In [ ]:
from random import randint
import tracemalloc
class Tracer:
    def __enter__(self):
        if tracemalloc.is_tracing():
            raise ValueError('nesting tracemalloc is not allowed')
        self.allocated = None
        tracemalloc.start()
        return self
    def __exit__(self, exc_type, exc_value, exc_traceback):
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.allocated = current

In [ ]:
N = 22
print(2**N/1_000_000, "млн.")

4.194304 млн.


In [5]:
from recordclass import dataobject

### Binary Tree

In [6]:
class Tree(dataobject):
    root: 'Node'

class Node(dataobject): 
    left: 'Node'
    right: 'Node'

class Leaf(dataobject):
    value: object

In [7]:
def add_nodes(depth):
    if depth == 0:
        return Leaf(randint(0,10))
    return Node(add_nodes(depth-1), add_nodes(depth-1))

In [19]:
class TreeGC(dataobject, gc=True):
   __fields__ = 'root',

class NodeGC(dataobject, gc=True): 
    __fields__ = 'left', 'right'

class LeafGC(dataobject, gc=True):
    __fields__ = 'value',

In [20]:
def add_nodes_gc(depth):
    if depth == 0:
        return LeafGC(randint(0,10))
    return NodeGC(add_nodes_gc(depth-1), add_nodes_gc(depth-1))

In [10]:
class TreeSlots:
    __slots__ = 'root',
    def __init__(self, root):
        self.root = root

class NodeSlots:
    __slots__ = 'left', 'right'
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
class LeafSlots:
    __slots__ = 'value',
    def __init__(self, value):
        self.value = value

In [11]:
def add_nodes_slots(depth):
    if depth == 0:
        return LeafSlots(randint(0,10))
    return NodeSlots(add_nodes_slots(depth-1), add_nodes_slots(depth-1))

In [12]:
with Tracer() as t1:
   tree = Tree(add_nodes(N))
print("dataobject:", t1.allocated // 1_000_000, 'Mb')
del tree, t1

dataobject: 234 Mb


In [13]:
%time tree = Tree(add_nodes(N))
%time del tree

CPU times: user 4.64 s, sys: 72 ms, total: 4.71 s
Wall time: 4.71 s
CPU times: user 149 ms, sys: 20 ms, total: 169 ms
Wall time: 168 ms


In [14]:
with Tracer() as t2:
   tree_slots = TreeSlots(add_nodes_slots(N))
print("__slots__:", t2.allocated // 1_000_000, 'Mb')
del tree_slots, t2

__slots__: 369 Mb


In [15]:
%time tree_slots = TreeSlots(add_nodes_slots(N))
%time del tree_slots

CPU times: user 15.2 s, sys: 136 ms, total: 15.3 s
Wall time: 15.3 s
CPU times: user 330 ms, sys: 24 ms, total: 354 ms
Wall time: 353 ms


In [16]:
with Tracer() as t3:
   tree_gc = TreeGC(add_nodes_gc(N))
print("dataobject+gc:", t3.allocated // 1000000, 'Mb')
del tree_gc, t3

dataobject+gc: 369 Mb


In [15]:
%time tree_gc = TreeGC(add_nodes_gc(N))
%time del tree_gc

CPU times: user 13.6 s, sys: 164 ms, total: 13.8 s
Wall time: 13.9 s
CPU times: user 252 ms, sys: 32 ms, total: 284 ms
Wall time: 283 ms


In [16]:
import sys
print('Size per instance:')
print('   node:', sys.getsizeof(Node(None,None)),
                  sys.getsizeof(NodeSlots(None,None)), 
                  sys.getsizeof(NodeGC(None,None)))
print('   leaf:', sys.getsizeof(Leaf(None)),
                  sys.getsizeof(LeafSlots(None)),
                  sys.getsizeof(LeafGC(None)))

Size per instance:
   node: 32 48 48
   leaf: 24 40 40


### General Tree

In [17]:
class GTree(dataobject, fast_new=True):
   __fields__ = 'root',

class GNode(dataobject, fast_new=True): 
    __fields__ = 'children',

class GLeaf(dataobject, fast_new=True):
    __fields__ = 'value',

In [18]:
def add_gnodes(depth):
    if depth == 0:
        return GLeaf(randint(0,10))
    children = litelist([])
    gnode = GNode(children)
    children.append(add_gnodes(depth-1))
    children.append(add_gnodes(depth-1))
    children.trim()
    return gnode

In [19]:
with Tracer() as gt1:
   gtree = GTree(add_gnodes(20))
print("dataobject:", gt1.allocated // 1000000, 'Mb')
del gtree, gt1

dataobject: 117 Mb


In [20]:
%time gtree = GTree(add_gnodes(20))
%time del gtree

CPU times: user 1.1 s, sys: 28 ms, total: 1.13 s
Wall time: 1.13 s
CPU times: user 54.7 ms, sys: 4.04 ms, total: 58.7 ms
Wall time: 58.6 ms


In [21]:
class GTreeSlots:
    __slots__ = 'root',
    def __init__(self, root):
        self.root = root

class GNodeSlots:
    __slots__ = 'children', 
    def __init__(self, children):
        self.children = children
    
class GLeafSlots:
    __slots__ = 'value',
    def __init__(self, value):
        self.value = value

In [22]:
def add_gnodes_slots(depth):
    if depth == 0:
        return GLeafSlots(randint(0,10))
    children = []
    gnode = GNodeSlots(children)
    children.append(add_gnodes_slots(depth-1))
    children.append(add_gnodes_slots(depth-1))
    return gnode

In [23]:
with Tracer() as gt2:
   gtree_slots = GTreeSlots(add_gnodes_slots(20))
print("__slots__:", gt2.allocated // 1000000, 'Mb')
del gtree_slots, gt2

__slots__: 176 Mb


In [24]:
%time gtree_slots = GTreeSlots(add_gnodes_slots(20))
%time del gtree_slots

CPU times: user 5.43 s, sys: 44.1 ms, total: 5.47 s
Wall time: 5.47 s
CPU times: user 144 ms, sys: 20 ms, total: 164 ms
Wall time: 163 ms
